In [1]:
import sys
import numpy
import pandas
import matplotlib
import seaborn
import scipy
import sklearn

print('Python: ()'.format(sys.version))
print('Numpy: ()'.format(numpy.__version__))
print('Pandas: ()'.format(pandas.__version__))
print('Matplotlib: ()'.format(matplotlib.__version__))
print('Seaborn: ()'.format(seaborn.__version__))
print('Scipy: ()'.format(scipy.__version__))
print('Sklearn: ()'.format(sklearn.__version__))

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# import the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the dataset from the csv file using pandas
data = pd.read_csv('D:\ALLDATA\TRAINING\ML\creditcard.csv')

In [ ]:
# Explore the dataset
print(data.columns)

In [ ]:
print(data.shape)

In [ ]:
print(data.describe())

In [ ]:
data= data.sample(frac=0.1, random_state=1)
print(data.shape)

In [ ]:
# Plot histogram of each parameter
data.hist(figsize=(20,20))
plt.show()

In [ ]:
# Determine number of fraud cases in dataset
Fraud = data[data['Class'] == 1]
Valid = data[data['Class'] == 0]
outlier_fraction = len(Fraud) / float(len(Valid))
print(outlier_fraction)

print('Fraud Cases: {}'.format(len(Fraud)))
print('Valid Cases: {}'.format(len(Valid)))

In [ ]:
# Correlation matrix
corrmat = data.corr()
fig = plt.figure(figsize = (12,9))

sns.heatmap(corrmat, vmax = .8, square = True)
plt.show()

In [ ]:
# Get all the columns from the DataFrame
columns = data.columns.tolist()

# Filter the columns to remove data we donot want
columns = [c for c in columns if c not in ["Class"]]

# Storage the variable we'll be predicting on
target = 'Class'

X = data[columns]
Y = data[target]

# Print the shapes of X and Y
print(X.shape)
print(Y.shape)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
# IsolationForest: Isolate the observation sample of max and min values
from sklearn.ensemble import IsolationForest
# LocalOutlierFactor: Unsupervised method, deviation of the neighbour
from sklearn.neighbors import LocalOutlierFactor

# define a random state
state = 1

# define the outlier detection methods
classifiers = {
    "Isolation Forest": IsolationForest(max_samples=len(X),
                                       contamination = outlier_fraction,
                                       random_state = state),
    "Local Outlier Factor": LocalOutlierFactor(
    n_neighbors = 20,
    contamination = outlier_fraction)
}

In [ ]:
# Fit the model
n_outlier = len(Fraud)

for i,(clf_name, clf) in enumerate(classifiers.items()):

# Fit the data and tag outliers
    if clf_name == "Local Outlier Factor":
        y_pred = clf.fit_predict(X)
        scores_pred = clf.negative_outlier_factor_
    else:
        clf.fit(X)
        scores_pred = clf.decision_function(X)
        y_pred = clf.predict(X)
        
    # Reshape the prediction value to 0 for valid, 1 for fraud
    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] = 1
    
    n_errors = (y_pred != Y).sum()
    
    # Run classification metrics
    print('{}:{}'.format(clf_name, n_errors))
    print(accuracy_score(Y, y_pred))
    print(classification_report(Y, y_pred))